In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [8]:
# Define image size and batch size
IMG_SIZE = (224, 224)  # Image size for VGG16
BATCH_SIZE = 32

# Paths to your dataset
train_dir = '/home/esidserver/datasets/testing_cls/train/'
val_dir = '/home/esidserver/datasets/testing_cls/val/'
test_dir = '/home/esidserver/datasets/testing_cls/test/'

In [9]:
# Data augmentation and rescaling
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load images from the directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=['AB', 'AG', 'ANO', 'CQ']  # Specify the class names
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=['AB', 'AG', 'ANO', 'CQ']  # Specify the class names
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=['AB', 'AG', 'ANO', 'CQ']  # Specify the class names
)


Found 5193 images belonging to 4 classes.
Found 1484 images belonging to 4 classes.
Found 740 images belonging to 4 classes.


In [10]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [13]:
# Load the VGG16 model pre-trained on ImageNet, excluding the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers (don't train them initially)
base_model.trainable = False

# Create a new model on top of it
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')  # 4 output classes
])

# Compile the model
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Set up a checkpoint to save the best model
checkpoint = ModelCheckpoint('best_vgg16_model.keras', monitor='val_accuracy', save_best_only=True, mode='max')

#Set up a earlystop to save the best model
early_stopping=EarlyStopping(monitor='val_accuracy', patience=50, restore_best_weights=True)

# Set up a checkpoint to save the best model
#checkpoint = ModelCheckpoint('best_vgg16_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')

In [14]:
#pip install --upgrade tensorflow tensorflow-gpu
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))



Num GPUs Available:  1


In [15]:
#auto early stop if the efficiecy is not improving for 50 epochs
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    epochs=250,  # Adjust the number of epochs based on your needs
    callbacks=[checkpoint]
)

# Unfreeze the base model for fine-tuning
base_model.trainable = True




Epoch 1/250


/home/esidserver/miniforge3/envs/tf/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1724404057.400978 2438733 service.cc:146] XLA service 0x7f712c00cd70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724404057.401040 2438733 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-08-23 16:07:37.485273: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-23 16:07:37.835723: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902


  1/162 ━━━━━━━━━━━━━━━━━━━━ 23:45 9s/step - accuracy: 0.2188 - loss: 1.8142

I0000 00:00:1724404061.910410 2438733 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


162/162 ━━━━━━━━━━━━━━━━━━━━ 78s 431ms/step - accuracy: 0.4748 - loss: 1.2573 - val_accuracy: 0.7677 - val_loss: 0.6398
Epoch 2/250
  1/162 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - accuracy: 0.6562 - loss: 0.8514

...

2024-08-23 18:11:08.797471: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 15192823614686788463
2024-08-23 18:11:08.797572: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 8637165143101458163
2024-08-23 18:11:08.819029: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 15192823614686788463
2024-08-23 18:11:08.819131: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 8637165143101458163


162/162 ━━━━━━━━━━━━━━━━━━━━ 58s 341ms/step - accuracy: 0.9360 - loss: 0.1817 - val_accuracy: 0.9708 - val_loss: 0.0874
Epoch 248/250
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 297us/step - accuracy: 0.9375 - loss: 0.1908 - val_accuracy: 0.9167 - val_loss: 0.2932
Epoch 249/250


2024-08-23 18:12:06.812585: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 8637165143101458163
2024-08-23 18:12:06.841609: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 15192823614686788463
2024-08-23 18:12:06.841704: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 8637165143101458163


162/162 ━━━━━━━━━━━━━━━━━━━━ 60s 354ms/step - accuracy: 0.9304 - loss: 0.1849 - val_accuracy: 0.9674 - val_loss: 0.0917
Epoch 250/250
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 354us/step - accuracy: 0.9375 - loss: 0.2061 - val_accuracy: 1.0000 - val_loss: 0.0307


2024-08-23 18:13:06.771712: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 15192823614686788463
2024-08-23 18:13:06.771822: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 8637165143101458163
2024-08-23 18:13:06.806101: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 15192823614686788463
2024-08-23 18:13:06.806207: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 8637165143101458163


In [18]:
# Recompile the model with a lower learning rate for fine-tuning
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
fine_tune_history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    epochs=50,  # Further epochs for fine-tuning
    callbacks=[checkpoint]
)


Epoch 1/50
162/162 ━━━━━━━━━━━━━━━━━━━━ 75s 401ms/step - accuracy: 0.8500 - loss: 0.3679 - val_accuracy: 0.9253 - val_loss: 0.1939
Epoch 2/50
  1/162 ━━━━━━━━━━━━━━━━━━━━ 18s 115ms/step - accuracy: 0.9375 - loss: 0.1496

2024-08-23 18:58:15.538768: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 15192823614686788463
2024-08-23 18:58:15.538903: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 8637165143101458163


162/162 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9375 - loss: 0.1496 - val_accuracy: 1.0000 - val_loss: 0.0770
Epoch 3/50
162/162 ━━━━━━━━━━━━━━━━━━━━ 66s 388ms/step - accuracy: 0.9060 - loss: 0.2245 - val_accuracy: 0.9436 - val_loss: 0.1585
Epoch 4/50
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 231us/step - accuracy: 0.9375 - loss: 0.1434 - val_accuracy: 0.9167 - val_loss: 0.3298
Epoch 5/50


2024-08-23 18:59:23.051752: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 15192823614686788463
2024-08-23 18:59:23.051882: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 8637165143101458163


162/162 ━━━━━━━━━━━━━━━━━━━━ 64s 380ms/step - accuracy: 0.9270 - loss: 0.1854 - val_accuracy: 0.9558 - val_loss: 0.1086
Epoch 6/50
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 240us/step - accuracy: 0.9375 - loss: 0.1742 - val_accuracy: 1.0000 - val_loss: 0.0781
Epoch 7/50


...

2024-08-23 19:20:34.025267: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 15192823614686788463
2024-08-23 19:20:34.025368: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 8637165143101458163


162/162 ━━━━━━━━━━━━━━━━━━━━ 59s 348ms/step - accuracy: 0.9920 - loss: 0.0224 - val_accuracy: 0.9952 - val_loss: 0.0135
Epoch 48/50
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 194us/step - accuracy: 1.0000 - loss: 0.0163 - val_accuracy: 0.9167 - val_loss: 0.2789
Epoch 49/50


2024-08-23 19:21:33.249252: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 15192823614686788463
2024-08-23 19:21:33.249343: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 8637165143101458163
2024-08-23 19:21:33.271153: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 15192823614686788463


162/162 ━━━━━━━━━━━━━━━━━━━━ 60s 353ms/step - accuracy: 0.9797 - loss: 0.0579 - val_accuracy: 0.9817 - val_loss: 0.0449
Epoch 50/50
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 264us/step - accuracy: 1.0000 - loss: 0.0057 - val_accuracy: 1.0000 - val_loss: 0.0019


2024-08-23 19:22:33.210121: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 15192823614686788463
2024-08-23 19:22:33.210248: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 8637165143101458163
2024-08-23 19:22:33.235586: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 15192823614686788463
2024-08-23 19:22:33.235697: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 8637165143101458163


In [19]:
# Load the best saved model
model.load_weights('best_vgg16_model.keras')

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_accuracy:.4f}")

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}")

47/47 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - accuracy: 0.9039 - loss: 0.2620
Validation Accuracy: 0.9063
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 128ms/step - accuracy: 0.9062 - loss: 0.2618
Test Accuracy: 0.9122
